In [4]:
import json
import sys
from pathlib import Path

import pandas as pd

with open('artefacts/parlamint_with_embeddings.json', 'r') as f:
    parlamint = json.load(f)
parlamint_es_ga = parlamint['ES-GA']
parlaming_gb = parlamint['GB']
parlamint_hu = parlamint['HU']
parlamint_ua = parlamint['UA']
parlamint_si = parlamint['SI']
parlamints = [parlamint_si, parlaming_gb, parlamint_es_ga, parlamint_hu, parlamint_ua]
parlamint_si[0].keys()
tf_idf_artefacts = Path('artefacts/tf_idf')

In [26]:
from bs4 import BeautifulSoup
import pandas as pd
import re


def embedding_path(speech_id):
    year = speech_id.split('_')[1].split('-')[0]
    language = speech_id.split('ParlaMint-')[1].split('_')[0]
    return f'artefacts/parlamint/{language}/{year}/{speech_id.split(".u")[0]}.json'


columns = ['id', 'embedding', 'tf_idf_score', 'n_sentences']
speech_scores = []
for parlamint in parlamints:
    print('processing', parlamint[0]['language'])
    with open(tf_idf_artefacts / f'{parlamint[0]["language"]}_sentence_weights.json', 'r') as f:
        sentece_weights = json.load(f)

    for document in parlamint:
        xml_file = document['xml_path']
        with open(xml_file, "r", encoding="utf8") as f:
            contents = f.read()
        soup = BeautifulSoup(contents, 'xml')
        embeddings = None
        n = 0
        for speech in soup.find_all('u'):
            n += 1
            speech_id = speech['xml:id']
            if not speech_id:
                continue
            if embeddings is None:
                with open(embedding_path(speech_id), 'r') as f:
                    embeddings = pd.read_json(f, orient='records', lines=True)
                    embeddings.set_index('sentence_id', inplace=True)
            speech_embeddings = []
            sentence_texts = []
            weights = []
            for sentence in speech.find_all('s'):
                sentence_id = sentence['xml:id']
                weights.append(sentece_weights[sentence_id])
                speech_embeddings.append(embeddings.loc[sentence_id])
                sentence_texts.append(re.sub(r'\s', ' ', sentence.get_text()))
            max_weight = max(weights)
            weights = [w / max_weight for w in weights]
            combined = sorted(zip(weights, sentence_texts), key=lambda x: x[0], reverse=True)
            for w, s in combined:
                print(w, s)
            print('-----')
            if n > 10:
                sys.exit(0)



processing SI
1.0  TOČKO DNEVNEGA REDA , TO JE NA PONOVNO ODLOČANJE O ZAKONU O SPREMEMBAH IN DOPOLNITVAH  ZAKONA O ZAPOSLITVENI REHABILITACIJI IN ZAPOSLOVANJU INVALIDOV  .                          
0.7578858706281392  Za obrazložitev zahteve dajem besedo predstavniku  Državnega sveta  gospodu  Alojzu Kovšci  .               
0.6615228504461174  Državni svet je na 12. izredni seji 22. decembra 2014 zahteval , da  Državni zbor  na podlagi tretje alineje prvega odstavka 97. člena  Ustave Republike Slovenije  pred razglasitvijo ponovno odloča o  Zakonu o spremembah  in dopolnitvah  Zakona o zaposlitveni rehabilitaciji in zaposlovanju invalidov  .                                               
0.5947517849170845  Obveščen sem , da se današnje seje ne more udeležiti naslednja poslanka in poslanci : gospa  Eva Irgl  , gospod  Andrej Čuš  , dr.  Franc Trček  , gospod  Franc Laj  , dr.  Bojan Dobovšek  , mag.  Branko Grims  , dr.  Matej T. Vatovec  , gospod  Roberto Battelli  in dr.  László Gön

SystemExit: 0

/home/vidklopcic/anaconda3/envs/dhh23/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
